# Script To Make Site

This script converts all jupyter notebooks into markdown files, then converts all markdown files (jupyter and note) into a website ready to push to github.

## Preliminaries

In [15]:
# Load libraries
import os
import re
import fileinput
import sys
from glob import glob
import shutil


# Create path to content
path = 'content/'

In [16]:
pwd

'/Users/piper/Desktop/github/ds_notes'

## Find All Jupyter Notebooks

In [17]:
# Find all jupyter notebooks in all content folders
all_ipynb_files = [os.path.join(root, name)
                   for root, dirs, files in os.walk(path)
                       for name in files
                           if name.endswith((".ipynb"))]

# Remove all notebooks from checkpoint folders
ipynb_files = [ x for x in all_ipynb_files if ".ipynb_checkpoints" not in x ]

## Convert All Jupyter Notebooks To Markdown

In [12]:
# For each file
for file in ipynb_files:
    # Convert into markdown
    os.system('jupyter nbconvert --to markdown {file}'.format(file=file))

## Handle The Folders

This really nasty code finds all the folders generated by the Juypter conversion (they end in `_files`) and then tries to rename them to remove the `_files` part (doing so is required by Hugo so the images can be seen). If an existing folder (without the `_files` part) exists, it moves all the files from the newly generated `_files` into the existing folder (so we retain any manual images like flashcard images in that existing folder) and then deletes the `_files` folder.

In [13]:
# Get all folders in directory
folders = [x[0] for x in os.walk(path)]

# Delete .ipynb checkpoint folders
folders = [ x for x in folders if ".ipynb_checkpoints" not in x ]

# For each folder
for folder_name in folders:
    # if _files in folder name
    if '_files' in folder_name:
        
        # Create a new folder name
        def rchop(thestring, ending):
            if thestring.endswith(ending):
                return thestring[:-len(ending)]
            return thestring

        new_folder_name = rchop(folder_name, '_files')
        
        # try to rename original folder
        try:
            os.rename(folder_name, new_folder_name)
            # if error, 
        except OSError:
            existing_base_folder = os.getcwd()+"/"+new_folder_name
            justcreated_base_folder = os.getcwd()+"/"+folder_name
                 
            # get a list of all files in the folder
            generated_files = os.listdir(justcreated_base_folder)
            
            # copy each file to the existing folder
            for generated_file in generated_files:
                to_copy = justcreated_base_folder+"/"+generated_file
                shutil.copy(to_copy, existing_base_folder)
  
            # delete the newly created _files folder
            shutil.rmtree(justcreated_base_folder)    

In [19]:
folders

['content/',
 'content/statistics',
 'content/statistics/frequentist',
 'content/statistics/frequentist/demonstrate_the_central_limit_theorem',
 'content/statistics/frequentist/probability_mass_functions',
 'content/statistics/frequentist/probability_mass_functions_files',
 'content/statistics/frequentist/demonstrate_the_central_limit_theorem_files',
 'content/postgresql',
 'content/postgresql/basics',
 'content/postgresql/tables',
 'content/postgresql/dates',
 'content/postgresql/add_delete_change',
 'content/postgresql/numeric',
 'content/postgresql/text',
 'content/postgresql/merging_and_joining',
 'content/python',
 'content/python/basics',
 'content/python/basics/set_the_color_of_a_matplotlib_files',
 'content/python/basics/set_the_color_of_a_matplotlib',
 'content/python/web_scraping',
 'content/python/data_wrangling',
 'content/python/data_wrangling/pandas_time_series_basics_files',
 'content/python/data_wrangling/pandas_normalize_column_files',
 'content/python/data_wrangling/p

## Search Through Contents Of All Markdown Files (manually generated and generated by a jupyter notebook) And Fix Links So That They Point To Folders Created Above

In [18]:
def replaceAll(file,searchExp,replaceExp):
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

# Find all markdown files in all content folders
all_md_files = [os.path.join(root, name)
               for root, dirs, files in os.walk(path)
               for name in files
               if name.endswith((".md"))]

for file in all_md_files:
    with open(file,'r') as f:
        filedata = f.read()
        # Find all markdown link syntaxes
        md_links = re.findall('!\\[[^\\]]+\\]\\([^)]+\\)', filedata)

        # For each markdown link
        for link in md_links:
            # Replace the full file path
            md_image_path = re.search(r'\((.*?)\)', link).group(1)
            md_image_filename = os.path.basename(md_image_path)
            md_image_title = re.search(r'\[(.*?)\]', link).group(1)

            new_link = "!["+md_image_title+"]("+md_image_filename+")"

            replaceAll(file, link, new_link)